In [ ]:
#To Dos:
#1) Investigate ratings that are being dropped (looks like pulling 26 ratings, but 30 of the other components)
#2) Attempt to use Selenium to pull full page (Should be closer to 50 rows per page)
#3) Attempt to pull additional pages
#3.5) Attempt to keep formatting (bolding/crossed-out text/italics) for ReviewerText
#4) Attempt to pull ReviewerProfile__Meta to provide the # of reviews by each reviewer
#5) If Selenium and beautiful soup fail, try below code option:

#Try below code to see if that will pull full page
#from requests_html import HTMLSession

#session = HTMLSession()
#r = session.get('https://www.pullandbear.com/rs/man/sale-c1030036006.html')
#r.html.render()

#print(r.html.html)

In [4]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from IPython.display import display, Image

In [5]:
driver = webdriver.Chrome()
title = driver.title

In [6]:
#response = requests.get('https://www.goodreads.com/book/show/59366244-i-m-glad-my-mom-died') 
for i in range(0,1000,100):
    #McCurdy - I'm Glad my Mom Died
    #response = requests.get('https://www.goodreads.com/book/show/59364173/reviews?reviewFilters={%22workId%22:%22kca://work/amzn1.gr.work.v3.JeHZlXvg2e1mD9_k%22,%22after%22:%22MjYwMTYsMTY2MDc1MjY5MjY2Mw%22}')
    #Hunger - Roxanne Gay
    response = requests.get('https://www.goodreads.com/book/show/26074156/reviews?reviewFilters={%22workId%22:%22kca://work/amzn1.gr.work.v1.FSsY8ohzUZCeEXoBsiEYqw%22,%22after%22:%22NjgxNSwxNTAwNjU3MjE4NDI1%22}')
print(response.status_code)

time.sleep(6)

doc = BeautifulSoup(response.text, 'html.parser')

200


In [7]:
df = pd.DataFrame(columns=['ReviewDate','ReviewerName','Rating','ReviewText','ReviewerMeta'])

In [8]:
book_tags_ = doc.find_all('article', class_ = "ReviewCard")
print(book_tags_[0])

<article aria-label="Review by Debbie" class="ReviewCard"><div class="ReviewCard__profile"><div class="ReviewerProfile ReviewerProfile--medium"><section class="ReviewerProfile__avatar"><a as="div" class="Avatar Avatar--medium" href="https://www.goodreads.com/user/show/8665845-debbie"><img alt="Profile Image for Debbie." class="Avatar__image" data-testid="image" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/users/1334166735i/8665845._UY200_CR33,0,200,200_.jpg"/></a></section><section class="ReviewerProfile__info"><span class="Text Text__title4"><div class="ReviewerProfile__name" data-testid="name"><a href="https://www.goodreads.com/user/show/8665845-debbie">Debbie</a></div></span><span class="Text Text__body3 Text__subdued"><div class="ReviewerProfile__meta"><span>479 reviews</span><span><span>3,622 followers</span></span></div></span></section><div class="ReviewerProfile__follow" data-testid="follow"></div></div></div><section class="ReviewCard__content"><section

In [9]:
#Working Copy- Ratings Pulls all 25 on Page; Next Steps iterate through rest of page
# Separate booktags for Ratings
book_tags_ = doc.find_all('span', class_ = "RatingStars RatingStars__small") # ["aria-label"])
print(book_tags_[0])


ratings = []

# Loop through all elements found
for tag in book_tags_:
    # Get the aria-label attribute from the current element
    aria_label = tag.get('aria-label')
    
    # Check if aria-label is not None and contains the expected format
    if aria_label and 'Rating ' in aria_label and ' out of 5' in aria_label:
        # Split the aria-label to extract the desired text
        rating_text = aria_label.split('Rating ')[1].split(' out of 5')[0]
        
        # Append the rating_text to the list of ratings
        ratings.append(rating_text)
    else:
        print(f"Skipping element with aria-label: {aria_label}")

# Create a dataframe from the list of ratings
df = pd.DataFrame({'Rating': ratings})

<span aria-label="Rating 3 out of 5" class="RatingStars RatingStars__small" role="img"><span class="baseClass RatingStar--small"><svg role="presentation" viewbox="0 0 24 24"><path class="RatingStar__fill" d="M24 9.63469C24 9.35683 23.7747 9.13158 23.4969 9.13158H15.0892L12.477 1.34327C12.4269 1.19375 12.3095 1.0764 12.16 1.02625C11.8966 0.937894 11.6114 1.07983 11.523 1.34327L8.91088 9.13158H0.503157C0.33975 9.13158 0.186521 9.21094 0.0922364 9.3444C-0.0680877 9.57134 -0.0140806 9.88529 0.212865 10.0456L7.00408 14.8432L4.40172 22.6166C4.35092 22.7683 4.37534 22.9352 4.46749 23.066C4.6275 23.2932 4.94137 23.3476 5.16853 23.1876L12 18.3758L18.8317 23.183C18.9625 23.2751 19.1293 23.2994 19.281 23.2486C19.5445 23.1604 19.6865 22.8752 19.5983 22.6117L16.996 14.8432L23.7872 10.0456C23.9206 9.95133 24 9.7981 24 9.63469Z"></path></svg></span><span class="baseClass RatingStar--small"><svg role="presentation" viewbox="0 0 24 24"><path class="RatingStar__fill" d="M24 9.63469C24 9.35683 23.7747 9.

In [10]:
#Working Copy- Reviewer Pulls all 25 on Page; Next Steps iterate through rest of page
# Separate booktags for ReviewerName
#Pulls 29 names
book_tags_ = doc.find_all('article', class_ = "ReviewCard") # ["aria-label"]) #ReviewCard
print(book_tags_[0])
#book_tags_ = doc.find_all('div', class_ = "ReviewerProfile__name")
#^Above code pulls below result as alternative approach
#<div class="ReviewerProfile__name" data-testid="name"><a href="https://www.goodreads.com/user/show/2190064-nenia-i-yeet-my-books-back-and-forth-campbell">Nenia ✨ I yeet my books back and forth ✨ Campbell</a></div>
#Could be more useful in terms of guaranteeing not doublecounting individuals by validating on user account URL

Reviewer = []

# Loop through all elements found
for tag in book_tags_:
    # Get the aria-label attribute from the current element
    aria_label = tag.get('aria-label')
    
    # Check if aria-label is not None and contains the expected format
    if aria_label and 'Review by ' in aria_label:
        # Split the aria-label to extract the desired text
        Reviewer_text = aria_label.split('Review by ')[1]
        
        # Append the Reviewer_text to the list of ratings
        Reviewer.append(Reviewer_text)
    else:
        print(f"Skipping element with aria-label: {aria_label}")

# Create a dataframe from the list of ratings
df = pd.DataFrame({'ReviewerName': Reviewer})

<article aria-label="Review by Debbie" class="ReviewCard"><div class="ReviewCard__profile"><div class="ReviewerProfile ReviewerProfile--medium"><section class="ReviewerProfile__avatar"><a as="div" class="Avatar Avatar--medium" href="https://www.goodreads.com/user/show/8665845-debbie"><img alt="Profile Image for Debbie." class="Avatar__image" data-testid="image" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/users/1334166735i/8665845._UY200_CR33,0,200,200_.jpg"/></a></section><section class="ReviewerProfile__info"><span class="Text Text__title4"><div class="ReviewerProfile__name" data-testid="name"><a href="https://www.goodreads.com/user/show/8665845-debbie">Debbie</a></div></span><span class="Text Text__body3 Text__subdued"><div class="ReviewerProfile__meta"><span>479 reviews</span><span><span>3,622 followers</span></span></div></span></section><div class="ReviewerProfile__follow" data-testid="follow"></div></div></div><section class="ReviewCard__content"><section

In [11]:
#Working Copy_Generating ReviewDate- Ratings Pulls all 25 on Page; Next Steps iterate through rest of page
#Pulls different count-Also looks at different pull of Book_tags_
reviewdt = []
book_tags_ = doc.find_all('span', class_="Text Text__body3")

# Initialize an empty list to store the text content
texts = []

# Loop through all elements found
for tag in book_tags_:
    # Get the text content from the current element
    reviewdt_content = tag.get_text(strip=True)
    
    # Append the text_content to the list of texts
    reviewdt.append(reviewdt_content)

# Create a dataframe from the list of texts
df = pd.DataFrame({'ReviewDate': reviewdt})

# Print or do further processing with the dataframe
print(df)


            ReviewDate
0       March 29, 2018
1     February 5, 2018
2      October 6, 2014
3        July 28, 2017
4       August 8, 2021
5        June 28, 2019
6        June 12, 2017
7    February 26, 2018
8        July 14, 2017
9     November 1, 2021
10    October 12, 2017
11       July 12, 2017
12        June 2, 2020
13    February 5, 2019
14       June 19, 2017
15      April 20, 2020
16       June 28, 2017
17  September 12, 2023
18     October 4, 2018
19       June 28, 2017
20      March 19, 2018
21       April 3, 2021
22      April 16, 2024
23   November 21, 2021
24       June 13, 2022
25      August 7, 2017
26      March 10, 2021
27     January 8, 2019
28    December 1, 2019
29       July 25, 2017


In [12]:
#Working Copy_Generating ReviewText- ReviewText Pulls all 25 on Page; Next Steps iterate through rest of page
#Pulls different count-Also looks at different pull of Book_tags_
#Can I clean the junk links from the text? or keep the format?
book_tags_ = doc.find_all('span', class_="Formatted")

# Initialize an empty list to store the text content
ReviewText = []

# Loop through all elements found
for tag in book_tags_:
    # Get the text content from the current element
    ReviewText_content = tag.get_text(strip=True)
    
    # Append the text_content to the list of texts
    ReviewText.append(ReviewText_content)

# Create a dataframe from the list of texts
df = pd.DataFrame({'ReviewText': ReviewText})

# Print or do further processing with the dataframe
print(df)


                                           ReviewText
0   Really torn about this one. On the one hand, t...
1   People see bodies like mine and make their ass...
2   I haven't written this yet but it will be okay...
3   This book is inoculated from too much criticis...
4   I cannot jump on the bandwagon of this being a...
5   “I do not want pity or appreciation or advice....
6   Beautifully written....Tender, poignant and co...
7   I want to give this a million stars. I want to...
8   I finishedHungerfive hours ago and still feel ...
9   The fact that this book exists is such a godda...
10  I am of two minds about this book.Firstly, Rox...
11  Listening to Roxanne Gay read her memoir, Hung...
12                      really hard & really powerful
13  This was a hard read. So important and so genu...
14  The thing I always admire about Roxane Gay's w...
15  In understated but moving prose, Roxane Gay re...
16  Roxane Gay is a National treasure.Hunger by Ro...
17                          

In [13]:
df = pd.DataFrame(columns=['ReviewDate','ReviewerName','Rating','ReviewText','ReviewerMeta'])

In [14]:
num_ratings = len(ratings)
review_dates = reviewdt[:num_ratings]
reviewer_names = Reviewer[:num_ratings]
review_texts = ReviewText[:num_ratings]
#reviewer_meta = reviewer_meta[:num_ratings]

In [15]:
#Fixing length of fields
num_ratings = len(ratings)
review_dates = reviewdt[:num_ratings]
reviewer_names = Reviewer[:num_ratings]
review_texts = ReviewText[:num_ratings]
#reviewer_meta = reviewer_meta[:num_ratings]

data_dict = {
    'ReviewDate': review_dates,
    'ReviewerName': reviewer_names,
    'Rating': ratings,
    'ReviewText': review_texts
    #'ReviewerMeta': reviewer_meta
}

In [16]:
test_df = pd.DataFrame(data_dict)
df = pd.concat([df, test_df], ignore_index=True)


#df. to_csv('McCurdy_Sample_Aug30.csv', index=False)
df. to_csv('Roxanne_Gay_Sample_Aug30.csv', index=False) 
